In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import missingno as msno
from collections import Counter

# Pre-proccessing

## Raw data:

In [60]:
data = pd.read_csv("./dataset/maps-synthetic-data-v1.1.csv", engine='python')
data.head()
#data.info()

,Unnamed: 0,X,flag,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,...,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1,1,synthetic,NaN,3.0,30,59.294132,181.602831,107.0,NaN,...,0.0,Yes,Yes,often,No,"Yes, Some Days",Other,"Yes, Every Day",Female,A
1,2,2,synthetic,NaN,9.0,26,NaN,NaN,85.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,B
2,3,3,synthetic,NaN,3.0,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Yes, Some Days","Yes, Every Day",Other,Male,A
3,4,4,synthetic,No,NaN,22,49.812426,160.224186,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Male,A
4,5,5,synthetic,No,8.0,31,62.270030,191.703227,132.0,Yes,...,NaN,NaN,NaN,often,No,"Yes, Some Days","Yes, Every Day","Yes, Every Day",Female,A


In [61]:
dictionary = pd.read_csv("./dataset/synthetic_data_dictionary.csv", engine='python')
dictionary.head()

,Variable Name,Variable ALSPAC Name,Variable Description,Variable Type,Administered to,Age of child at administration,ALSPAC Measure,ALSPAC Responses,Processing Details,Reference Document
0,agg_score,f596,Aggression score of partnership,Discrete,Mother,Aged 8 months,Derived by ALSPAC. See Processing Details,NaN,Aggression score was determined by three quest...,NaN
1,alon_week,ccs1006,Average time child spent per day doing things ...,Ordinal,Child,Aged 198 months,"""How much time on average do you spend each da...",Not at all,"The category ""Not at all"" was collapsed into t...",NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Less than 1 hour,NaN,NaN
3,,,NaN,NaN,NaN,NaN,NaN,1-2 hours,NaN,NaN
4,,,NaN,NaN,NaN,NaN,NaN,3 or more hours,NaN,NaN


Variables have different Age Of Child At Administration parameters. We don't need the variables with this parameter not equal to 16 (for variables related to screen time) and 18 (for variables related to a depression or anxiety diagnosis).

## only the columns at 16 and 18:

All vars at 16 are separated into weekend and weekdays. Might be worth summing it up into total per week.

In [62]:
rows = dictionary[(dictionary["Age of child at administration"] == "Aged 198 months")]
rows2 = dictionary[(dictionary["Age of child at administration"] == "Aged around 17.5 years")]
rows3 = dictionary[(dictionary["Age of child at administration"] == "Aged around 15.5 years")]

columns_to_keep_names = list(rows['Variable Name'].append(rows2['Variable Name']).append(rows3['Variable Name']))
columns_to_keep_names.append('iq')
#print(columns_to_keep_names, len(columns_to_keep_names))

C:\Users\moham\AppData\Local\Temp/ipykernel_28652/655318495.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  columns_to_keep_names = list(rows['Variable Name'].append(rows2['Variable Name']).append(rows3['Variable Name']))
C:\Users\moham\AppData\Local\Temp/ipykernel_28652/655318495.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  columns_to_keep_names = list(rows['Variable Name'].append(rows2['Variable Name']).append(rows3['Variable Name']))


## Changing the data types:
Changing the categories to numbers and yes/no answers to 0/1

In [63]:
for i in ['comp_bed_9', 'pat_pres_10', 'pat_pres_8', 'pat_pres', 'mat_anx_1', 'mat_anx_18m', 'mat_anx_8m', 'emot_cruel',
          'phys_cruel', 'mat_anx_0m', 'child_bull', 'musi_13', 'tv_bed_9', 'own_mob', 'comp_games']:
    data[i] = data[i].replace(['Yes'], 1)
    data[i] = data[i].replace(['No'], 0)
    
for i in ['comp_noint_bed_16', 'comp_int_bed_16', 'comp_house', 'tv_bed_16']:
    data[i] = data[i].replace(['Yes'], 1)
    data[i] = data[i].fillna(0)
    
for i in ['talk_phon_wend', 'musi_wend', 'talk_phon_week', 'musi_week']:
    data[i] = data[i].replace(['Not at all'], 0)
    data[i] = data[i].replace(['Any at all'], 1)

for i in ['play_wend', 'out_sum_wend', 'play_week', 'out_sum_week']:
    data[i] = data[i].replace(['Less than 3 hours'], 0)
    data[i] = data[i].replace(['3 or more hours'], 3)
    
for i in ['tran_wend', 'tran_week']:
    data[i] = data[i].replace(['Less than 1 hour'], 0)
    data[i] = data[i].replace(['1 or more hours'], 1)


In [64]:
for i in ['talk_mob_wend', 'read_wend', 'draw_wend', 'talk_mob_week', 'read_week', 'draw_week']:
    data[i] = data[i].replace(['Not at all', 'Less than 1 hour', '1 or more hours'], range(3))
    
for i in ['work_wend', 'alon_wend', 'out_win_wend', 'work_week', 'alon_week', 'out_win_week']:
    data[i] = data[i].replace(['Less than 1 hour', '1-2 hours', '3 or more hours'], [0, 1, 3])
    
for i in ['phone_14_wend', 'phone_14_week']:
    data[i] = data[i].replace(['Not at all', '< 1 hour', '1 or more hours'], range(3))
    
for i in ['fam_tv_eve', 'fam_tv_aft', 'fam_tv_mor']:
    data[i] = data[i].replace(['Other', 'Yes, Some Days', 'Yes, Every Day'], range(3))    

In [65]:
for i in ['text_wend', 'comp_wend', 'tv_wend', 'text_week', 'comp_week', 'tv_week']:
    data[i] = data[i].replace(['Not at all', 'Less than 1 hour', '1-2 hours', '3 or more hours'], range(4))

In [66]:
percentages = ['dep_band_07', 'dep_band_15', 'dep_band_13', 'dep_band_10', 'anx_band_15', 'anx_band_07',
               'anx_band_13', 'anx_band_10']
for i in percentages:
    data[i] = data[i].str.strip('%<>~').astype('float')

In [67]:
data['weight_16'].round(decimals = 2)
data['height_16'].round(decimals = 2).head()
data['has_dep_diag'].replace(['Yes ICD-10 diagnosis of depression', ' No ICD-10 diagnosis of depression'], [1, 0], inplace = True)
data['exercise'].replace(['Never','Less than once a month', '1-3 times a month',
                         '1-4 times a week', '5 or more times a week'], range(5), inplace = True)

## Remove vars at wrong ages

In [68]:
data = pd.DataFrame(data)
for i in data.columns:
    if (i not in columns_to_keep_names):
        data.drop(i, inplace = True, axis = 1)

## Combining variables
Weekend + weekday columns combines into a sum column

In [69]:
add1 = ['alon_week','draw_week','musi_week','out_sum_week','out_win_week', 'play_week','read_week','text_week',
        'tran_week','work_week','comp_week','tv_week','talk_mob_week','talk_phon_week', 'comp_int_bed_16']
add2 = ['alon_wend','draw_wend','musi_wend','out_sum_wend','out_win_wend', 'play_wend','read_wend','text_wend',
        'tran_wend','work_wend','comp_wend','tv_wend','talk_mob_wend','talk_phon_wend', 'comp_noint_bed_16']
add3 = ['alon','draw','musi','out_sum','out_win', 'play','read','text',
        'tran','work','comp','tv','talk_mob','talk_phon', 'comp_bed_16']

for i in range(len(add1)):
    data[add3[i]] = data[add1[i]] + data[add2[i]]

data['comp_house_at_all'] = (data['comp_int_bed_16'] + data['comp_noint_bed_16'] + data['comp_house'])
for i in ['talk_phon', 'musi']:
    data[i].replace([2], 1, inplace = True)

## Checking if combined variables were answered together


In [70]:
error = []
for i in data.index:
    for j in range(len(add1)):
        if(type(data[add1[j]][i]) != type(data[add2[j]][i])):
            if(add1[j] not in error):
                error.append(add1[j])

for i in error:
    print(i , data[i].unique())

## Export the pre-proccessed data:

In [71]:
data.to_csv("newdata2.csv") # all vars

In [72]:
#refined variables
remove = np.append(add1, add2)
data2 = data.drop(remove, axis = 1)

In [73]:
data2.to_csv('newdata3.csv')